# Object-oriented FISSA interface

This notebook contains a step-by-step example of how to use the object-oriented (class-based) interface to the [FISSA](https://github.com/rochefort-lab/fissa) toolbox.

For more details about the methodology behind FISSA, please see our paper:
> S. W. Keemink, S. C. Lowe, J. M. P. Pakan, E. Dylda, M. C. W. van Rossum, and N. L. Rochefort. FISSA: A neuropil decontamination toolbox for calcium imaging signals, *Scientific Reports*, **8**(1):3493, 2018. doi: [10.1038/s41598-018-21640-2](https://www.doi.org/10.1038/s41598-018-21640-2).

See [basic_usage.py](https://github.com/rochefort-lab/fissa/blob/master/examples/basic_usage.py) (or [basic_usage_windows.py](https://github.com/rochefort-lab/fissa/blob/master/examples/basic_usage_windows.py) for Windows users) for a short example script outside of a notebook interface.

In [ ]:
# Import the FISSA toolbox
import fissa

In this notebook, we will be plotting our results using [HoloViews](http://holoviews.org/), but you can use any plotting library you prefer ([matplotlib](https://matplotlib.org/), [bokeh](https://docs.bokeh.org/), etc).

In [ ]:
# Import our plotting toolbox, and enable options for embedded notebook figures
import holoviews as hv

%load_ext holoviews.ipython
%output widgets="embed"

## Defining an experiment

To run a separation step with fissa, you need create a `fissa.Experiment` object, which will hold your extraction parameters and results.

The inputs to the `fissa.Experiment` instance are the:
- experiment images
- the regions of interest (ROIs) to extract
Define your inputs. All that's necessary to define are the image data and ROIs. 

Images can be defined as a folder with tiff stacks:
```python
images = "folder"
```
Where each tiff stack in the folder is a trial with several frames.

Or the data can also be given as a list of arrays if not stored as tiffs:
```python
images = [array1, array2, array3, ...]
```

For ROIs either a set of ROIs across all images should be defined, or a set of ROIs per image. 

If the ROIs were defined using ImageJ use ImageJ's export function to save them in a zip. Then, indicate the locations as a list:
```python
rois = "rois.zip"  # for a single set of rois across images
rois = ["rois1.zip", "rois2.zip", ...]  # for a roiset for each image
```
Defining a different roiset per image can be useful if you need to adjust for motion drift for example.

Then, we can define our experiment:

In [ ]:
# Define the folder where FISSA's outputs will be cached, so they can be
# quickly reloaded in the future without having to recompute them.
#
# This argument is optional; if it is not provided, FISSA will not save its
# results for later use.
#
# Note: you *must* use a different folder for each experiment,
# otherwise FISSA will load the in the folder provided instead
# of computing results for the new experiment.
#
# In this example, we will use the current datetime as the
# name of the experiment, but you can name your experiments
# however you want to.

import datetime

output_folder = "fissa-example_{}".format(
    datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
)

print(output_folder)

In [ ]:
# Define image and ROI locations
images_location = "exampleData/20150529"
rois_location = "exampleData/20150429.zip"

experiment = fissa.Experiment(images_location, rois_location, output_folder)

Previously analyzed experiments in output_folder will be loaded, if they exist, and the next step could be skipped.

### Extracting traces and separating them
Next, we need to extract the traces and separate them:

In [ ]:
experiment.separate()

If you want to redo preparation and/or separation you can set:
```
experiment.separate(redo_prep=True, redo_sep=True)
```
(If you redo prepartion this will also redo the separation, to make sure these always match up).

## Accessing results

After running ```experiment.separate()``` the results are stored as follows.

### ROI outlines

The ROI outlines, as well as the extra neuropil regions, can be found as in ```experiment.roi_polys``` as follows. For cell number ```c``` and tiff number `t`, the set of ROIs for that cell and tiff is at
```python
experiment.roi_polys[c][t][0][0]  # basic ROI
experiment.roi_polys[c][t][n][0]  # n = 1, 2, 3, .... the neuropil regions
```
Sometimes ROIs cannot be expressed as a single polygon (e.g. a ring-ROI), in those cases several polygons are used to describe it as:
```python
experiment.roi_polys[c][t][n][i]  # i iterates over the different polygons
```

As an example, plotting the first region of interest plus its surrounding neuropil subregions.

In [ ]:
# Visualise the ROI polygon for a sample cell
c = 0
t = 0

neuropil1 = hv.Curve(experiment.roi_polys[c][t][1][0])
neuropil2 = hv.Curve(experiment.roi_polys[c][t][2][0])
neuropil3 = hv.Curve(experiment.roi_polys[c][t][3][0])
neuropil4 = hv.Curve(experiment.roi_polys[c][t][4][0])
cell = hv.Curve(experiment.roi_polys[c][t][0][0])

neuropil1 * neuropil2 * neuropil3 * neuropil4 * cell

In this example, the ROI (grey) is on the boundary of the image, so the surrounding neuropil subregions are arranged to share the available space.

### FISSA extracted traces

The final extracted traces can be found in ```experiment.result``` as follows. For cell number ```c``` and tiff number `t`, the final extracted trace is given by:
```python
experiment.result[c][t][0, :]
```

In `experiment.result` one can find the signals present in the cell ROI, ordered by how strongly they are present (relative to the surrounding regions). `experiment.result[c][t][0, :]` gives the most strongly present signal, and is considered the cell's "true" signal. `[i, :]` for `i=1,2,3,...` gives the other signals which are present in the ROI, but driven by other cells or neuropil.

### Before decontamination

The raw extracted signals can be found in `experiment.raw` in the same way. Now in  `experiment.raw[c][t][i,:]`, `i` indicates the region number, with `i=0` being the cell, and `i=1,2,3,...` indicating the surrounding regions.

As an example, plotting the raw and extracted signals for the second trial for the third cell:

In [ ]:
c = 2
t = 1

(
    hv.Curve(experiment.raw[c][t][0, :], label="Raw")
    * hv.Curve(experiment.result[c, t][0, :], label="Decontaminated")
)

### df/f<sub>0</sub>

It is often useful to calculate the intensity of a signal relative to the baseline value, df/f<sub>0</sub>, for the traces.
This can be done as follows.

Note that by default, f<sub>0</sub> is determined as the minimum across all trials (all tiffs) to ensure that results are directly comparable between trials, but you can normalise each trial individually instead if you prefer by setting `across_trials=False`.

Since FISSA is very good at removing contamination from the ROI signals, the minimum value on the decontaminated trace will typically be `0.`. Consequently, we use the minimum value of the (smoothed) raw signal to provide the f<sub>0</sub> from the raw trace for both the raw and decontaminated df/f<sub>0</sub>.

In [ ]:
experiment.calc_deltaf(freq=10, across_trials=True)

In [ ]:
c = 2
t = 1

(
    hv.Curve(experiment.deltaf_raw[c][t][0, :], label="Raw", vdims=["df/f0"])
    * hv.Curve(experiment.deltaf_result[c, t][0, :], label="Decontaminated")
)

## Exporting to MATLAB

The results can easily be exported to a MATLAB-compatible matfile as follows.

The output will appear in the `output_folder` we supplied to `experiment` when we created it.

In [ ]:
experiment.save_to_matlab()

Loading `output_folder/matlab.mat` in MATLAB will give you three structs, `ROIs`, `raw`, and `result`.

These interface similarly as `experiment.ROIs`, `experiment.raw`, and `experiment.result` described above. However, Matlab counts from 1 (as opposed to Python counting from 0), such that the ROI, raw trace, and decontaminated trace are all found for cell 0 trial 0 as:

```octave
ROIs.cell0.trial0{1}  % polygon for the ROI
ROIs.cell0.trial0{2}  % polygon for first neuropil subregion
result.cell0.trial0(1, :)  % final extracted cell signal
result.cell0.trial0(2, :)  % contaminating signal
raw.cell0.trial0(1, :)  % raw measured celll signal
raw.cell0.trial0(2, :)  % raw signal from first neuropil subregion
```

## Addendum

### Finding the tiff files

If you find something noteworthy in one of the traces and need to backreference to the corresponding tiff file, you can look up the path to the tiff file with `experiment.images`.

In [ ]:
trial_of_interest = 1

print(experiment.images[trial_of_interest])

### Mean image data

You can get the temporal-mean image for a trial with `experiment.means`.

In [ ]:
hv.Image(experiment.means[trial_of_interest])

You can also superimpose the cell ROIs on the temporal-mean image as follows.

In [ ]:
# Using holoviews

t = trial_of_interest

fig = hv.Raster(experiment.means[t])

for c in range(experiment.nCell):
    roi_poly = experiment.roi_polys[c][t][0][0]
    x = roi_poly[:, 1]
    y = roi_poly[:, 0]
    fig *= hv.Curve(zip(x, y))

fig

In [ ]:
# Using matplotlib

import matplotlib.pyplot as plt

t = trial_of_interest

plt.imshow(experiment.means[t], cmap="gray")
for roi_poly in experiment.roi_polys:
    # Plot border around cells
    # plt.plot(roi_poly[t][0][0][:, 1], roi_poly[t][0][0][:, 0], ":c")
    # Fill cells with partially-transparent shaded area
    plt.fill(roi_poly[t][0][0][:, 1], roi_poly[t][0][0][:, 0], ":r", alpha=0.2)
plt.show()

### FISSA customisation settings

FISSA has several user-definable settings, which can be set when defining the `experiment` object.

In [ ]:
# FISSA uses multiprocessing to speed up its processing.
# By default, it will spawn one worker per CPU core on your machine.
# However, if you have a lot of cores and not much memory, you many not
# be able to suport so many workers simultaneously.
# In particular, this can be problematic during the data preparation step
# in which tiffs are loaded into memory.
# The default number of cores for the data preparation and separation steps
# can be changed as follows.
ncores_preparation = 4  # If None, uses all available cores
ncores_separation = None  # if None, uses all available cores

# By default, FISSA uses 4 subregions for the neuropil region.
# If you have very dense data with a lot of different signals per unit area,
# you may wish to increase the number of regions.
nRegions = 8

# By default, each surrounding region has the same area as the central ROI.
# i.e. expansion = 1
# However, you may wish to increase or decrease this value.
expansion = 0.75

# The degree of signal sparsity can be controlled with the alpha parameter.
alpha = 0.1

# Set up a FISSA experiment with these parameters
experiment = fissa.Experiment(
    images_location,
    rois_location,
    output_folder,
    nRegions=nRegions,
    expansion=expansion,
    alpha=alpha,
    ncores_preparation=ncores_preparation,
    ncores_separation=ncores_separation,
)
# Extract the data with these new parameters.
# Note that we are using the same output folder as before. Since FISSA has cached
# a result alrady to this directory, its default behaviour is to restore the
# previously generated results.
# To make sure FISSA runs a fresh decontamination process with the new parameters,
# we need to make sure to specify to redo the preparation and separation.
# FISSA will then ignore the cached output and overwrite it with new results.
experiment.separate(redo_prep=True)

We can plot the new results for our example trace from before. Although we doubled the number of neuropil regions around the cell, very little has changed for this example because there were not many sources of contamination.

However, there will be more of a difference if your data has more neuropil sources per unit area within the image.

In [ ]:
# Plot the new results
c = 2
t = 1

(
    hv.Curve(experiment.raw[c][t][0, :], label="Raw")
    * hv.Curve(experiment.result[c, t][0, :], label="Decontaminated")
)

Alternatively, these settings can be refined after creating the `experiment` object, as follows.

In [ ]:
experiment.ncores_preparation = 8
experiment.alpha = 0.1
experiment.expansion = 0.75

### Loading data from large tiff files

By default, FISSA loads entire tiff files into memory at once and then manipulates all ROIs within the tiff.
This can sometimes be problematic when working with very large tiff files which can not be loaded into memory all at once.
If you have out-of-memory problems, you can activate FISSA's low memory mode, which will cause it to manipulate each tiff file frame-by-frame.

In [ ]:
experiment = fissa.Experiment(
    images_location, rois_location, output_folder, lowmemory_mode=True
)
experiment.separate(redo_prep=True)

### Handling custom formats

By default, FISSA can use tiff files or numpy arrays as its input image data, and numpy arrays or ImageJ zip files for the ROI definitions.
However, it is also possible to extend this functionality and integrate other data formats into FISSA in order to work with other custom and/or proprietary formats that might be used in your lab.

This is done by defining your own DataHandler class.
Your custom data handler should be a subclass of [fissa.extraction.DataHandlerAbstract](https://fissa.readthedocs.io/en/stable/source/packages/fissa.extraction.html#fissa.extraction.DataHandlerAbstract), and implement the following methods:

- `image2array(image)` takes an image of whatever format and turns it into *data* (typically a numpy.ndarray).
- `getmean(data)` calculates the 2D mean for a video defined by *data*.
- `rois2masks(rois, data)` creates masks from the rois inputs, of appropriate size *data*.
- `extracttraces(data, masks)` applies the *masks* to *data* in order to extract traces.

See [fissa.extraction.DataHandlerAbstract](https://fissa.readthedocs.io/en/stable/source/packages/fissa.extraction.html#fissa.extraction.DataHandlerAbstract) for further description for each of the methods.

If you only need to handle a new *image* input format, which is converted to a numpy.ndarray, you may find it is easier to create a subclass of the default datahandler, [fissa.extraction.DataHandlerTifffile](https://fissa.readthedocs.io/en/stable/source/packages/fissa.extraction.html#fissa.extraction.DataHandlerTifffile).
In this case, only the `image2array` method needs to be overwritten and the other methods can be left as they are.

In [ ]:
from fissa.extraction import DataHandlerTifffile

# Define a custom datahandler class.
#
# By inheriting from DataHandlerTifffile, most methods are defined
# appropriately. In this case, we only need to overwrite the
# `image2array` method to work with our custom data format.


class DataHandlerCustom(DataHandlerTifffile):
    @staticmethod
    def image2array(image):
        """Open a given image file as a custom instance.

        Parameters
        ----------
        image : custom
            Your image format (avi, hdf5, etc.)

        Returns
        -------
        numpy.ndarray
            A 3D array containing the data, shaped
            ``(frames, y_coordinate, x_coordinate)``.
        """
        # Some custom code
        pass


# Then pass an instance of this class to fissa.Experiment when creating
# a new experiment.
datahandler = DataHandlerCustom()

experiment = fissa.Experiment(
    images_location,
    rois_location,
    datahandler=datahandler,
)

For advanced users that want to entirely replace the DataHandler with their own methods, you can also inherit a class from the abstract class, [fissa.extraction.DataHandlerAbstract](https://fissa.readthedocs.io/en/stable/source/packages/fissa.extraction.html#fissa.extraction.DataHandlerAbstract).
This can be useful if you want to integrate FISSA into your workflow without changing everything into the numpy array formats that FISSA usually uses internally.